In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
retail = pd.read_csv('retail_clean (1).csv')
retail = retail.drop(columns='Unnamed: 0')
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


In [3]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779495 entries, 0 to 779494
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      779495 non-null  int64  
 1   StockCode    779495 non-null  object 
 2   Description  779495 non-null  object 
 3   Quantity     779495 non-null  int64  
 4   InvoiceDate  779495 non-null  object 
 5   Price        779495 non-null  float64
 6   Customer ID  779495 non-null  float64
 7   Country      779495 non-null  object 
 8   Revenue      779495 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 53.5+ MB


In [4]:
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'])

In [5]:
retail['daysofweek'] = retail['InvoiceDate'].dt.dayofweek

In [6]:
retail['daysofweek'].value_counts
retail['date'] = retail['InvoiceDate'].dt.strftime('%Y-%m-%d')
retail['date'] = pd.to_datetime(retail['date'])

# GETTING THE CV^2 AND ADI

In [7]:
retail_grouped = retail.groupby(['Description','date']).agg(total_sales = ('Quantity','sum')).reset_index()

In [8]:
cv_data = retail_grouped.groupby('Description').agg(average = ('total_sales','mean'),
                                                   standard_dev = ('total_sales','std')).reset_index()

In [9]:
cv_data['cv_squared'] = (cv_data['standard_dev']/cv_data['average'])**2

In [10]:
product_by_date = retail.groupby(['Description','date']).agg(count=('Description','count')).reset_index()

In [11]:
cv_data

,Description,average,standard_dev,cv_squared
0,DOORMAT UNION JACK GUNS AND ROSES,6.958333,7.865606,1.277774
1,3 STRIPEY MICE FELTCRAFT,11.220339,14.247889,1.612460
2,4 PURPLE FLOCK DINNER CANDLES,7.363636,10.850123,2.171124
3,50'S CHRISTMAS GIFT BAG LARGE,31.416667,55.502112,3.121036
4,ANIMAL STICKERS,32.083333,11.758620,0.134324
...,...,...,...,...
5278,ZINC T-LIGHT HOLDER STARS SMALL,36.522388,46.825017,1.643757
5279,ZINC TOP 2 DOOR WOODEN SHELF,5.902439,7.598700,1.657355
5280,ZINC WILLIE WINKIE CANDLE STICK,21.116438,23.437864,1.231955
5281,ZINC WIRE KITCHEN ORGANISER,2.500000,2.505549,1.004444


In [12]:
skus = product_by_date.Description.unique()

In [13]:
skus

array(['  DOORMAT UNION JACK GUNS AND ROSES', ' 3 STRIPEY MICE FELTCRAFT',
       ' 4 PURPLE FLOCK DINNER CANDLES', ...,
       'ZINC WILLIE WINKIE  CANDLE STICK', 'ZINC WIRE KITCHEN ORGANISER',
       'ZINC WIRE SWEETHEART LETTER TRAY'], dtype=object)

In [14]:
empty_dataframe = pd.DataFrame()

In [16]:
skus.shape

(5283,)

In [ ]:
for sku in skus:
    a = product_by_date[product_by_date.Descripton == sku]
    a['previous_date'] = a['date'].shift(1)
    empty_dataframe = pd.concat([empty_dataframe,a],axis=0)

In [ ]:
empty_dataframe['Duration'] = empty_dataframe['date'] - empty_dataframe['previous_date']